# Agenda

## Create a kubernetes cluster with VMware Enterprise PKS

## Attach the cluster to VMware Tanzu Mission Control
**Authenticate to the new K8s cluster**

**Verify connection to the K8s cluster**

## Login to VMware Tanzu Mission Control
### Authenticate via tmc CLI
### query cluster groups and clusters
### Attach K8s cluster to Tanzu Mission Control
___

## Create PKS Kubernetes Cluster

### Authenticate to your PKS API and review existing clusters

In [ ]:
!pks

In [6]:
!export TS_G_ENV=poway
!export UAA_ADMIN_PASSWORD=$(jq -r < ~/Downloads/$TS_G_ENV.json .pks_api.uaa_admin_password)
!pks login -a api.pks.$TS_G_ENV.cf-app.com -u admin -p $UAA_ADMIN_PASSWORD -k

/bin/sh: /Users/khan/Downloads/.json: No such file or directory

Error: Post https://api.pks..cf-app.com:8443/oauth/token: dial tcp: lookup api.pks..cf-app.com: no such host


In [8]:
!pks clusters


Name                 Plan Name  UUID                                  Status     Action
poway-pks-cluster-1  small      32831344-445d-43a4-9502-efa8832a973d  succeeded  CREATE
photon-pks           small      bd479180-3e31-4e78-889a-8c20363d360b  succeeded  CREATE



*Example Output:*

```
Name        Plan Name  UUID                                  Status     Action
test-demo   small      77e8836c-1849-46ed-999c-26be9672bc0e  succeeded  CREATE
```

In [ ]:
pks cluster test-demo

In [ ]:
pks plans

### Create the PKS cluster

In [ ]:
pks create-cluster

### Decision time... 

Decide now how you will access the master node(s) of the resulting K8s cluster before you continue.

In my case, I have a domain us.azure.bekind.io into which I will manually create a DNS entry for a load balancer whose backend points to a pool of master node(s).

At the very least, if you specify an IP address in the "external-hostname," you have to have already acquired that public IP address on your cloud provider.  This IP or DNS name should be publically accessible.

In [ ]:
!pks create-cluster pa-jkhan-compton --external-hostname mytmc-cluster-pa-jkhan --plan small

**Example Output:**
```
Name:                     test-demo
Plan Name:                small
UUID:                     7c2f04bc-a66e-4f7b-a9a4-79574bebb470
Last Action:              CREATE
Last Action State:        in progress
Last Action Description:  Creating cluster
Kubernetes Master Host:   test.us.azure.bekind.io
Kubernetes Master Port:   8443
Worker Nodes:             3
Kubernetes Master IP(s):  In Progress
Network Profile Name:

Use 'pks cluster test-azure' to monitor the state of your cluster
```

Monitor `pks clusters` until the cluster creation is successful

In [9]:
!pks clusters


Name                 Plan Name  UUID                                  Status     Action
poway-pks-cluster-1  small      32831344-445d-43a4-9502-efa8832a973d  succeeded  CREATE
photon-pks           small      bd479180-3e31-4e78-889a-8c20363d360b  succeeded  CREATE



Capture the credentials for your new K8s cluster

In [ ]:
!pks get-credentials compton-pks-cluster-1

Examine your new cluster

In [ ]:
!kubectl config use-context compton-pks-cluster-1

In [ ]:
!kubectl cluster-info

In [10]:
!kubectl get namespaces

NAME                STATUS   AGE
default             Active   6d23h
kube-node-lease     Active   6d23h
kube-public         Active   6d23h
kube-system         Active   6d23h
pa-jkhan-ns         Active   6d20h
pks-system          Active   6d23h
vmware-system-tmc   Active   6d21h


# Attach the cluster to Tanzu Mission Control

Login to TMC via https://console.cloud.vmware.com

Download the TMC CLI, make it executable, and move it into your system path

*Cluster names and environment variables*

In [ ]:
!export TMC_CLUSTER_GROUP=pa-jkhan-poway
!export TMC_CLUSTER=pa-jkhan-poway

In [ ]:
!tmc

In [4]:
!tmc login --name jkhan --no-configure

ℹ To fetch an API token visit https://console.cloud.vmware.com/csp/gateway/portal/#/user/tokens 
✔ API token: █

Error: ^C
Usage:
  tmc login [flags]
  tmc login [action]

Examples:
  # Interactive login.
  tmc login

  # Environment variable login (note: set $API_TOKEN beforehand)
  tmc login --name <context name>
  
  # Only login, no default environemnt configuration.
  tmc login --no-configure
  
  # Switch login contexts.
  tmc login switch

Actions:
  switch      Switch login to another existing context

Flags:
  -h, --help           help for login
  -n, --name string    name for the login context, will be generated if not provided
  -c, --no-configure   just login, do not configure the environment

Use "tmc login [command] --help" for more information about a command.

✗ ^C 


**Example Output:**
```
ℹ To fetch an API token visit https://console.cloud.vmware.com/csp/gateway/portal/#/user/tokens
API token****************************************************************
Login context name  VMW_W1
? Select default log level info
? Select default credential tmc-workshop
? Select default region us-east-1
? Select default AWS SSH key bjung@vmware.com
✔ Successfully created context "VMW_W1", to manage your contexts run `tmc system context -h`
```

In [11]:
!tmc clustergroup list

  NAME                      LABELS                       
  akadari                   origin:akadari               
  area51                    origin:atari                 
  arul-tmc                                               
  astroids                  origin:atari                 
  avannala                                               
  bboe                                                   
  bragazzi-pa-pks                                        
  caas-workshop             team:dell-tech               
  cdelashmutt                                            
  centipede                 origin:atari                 
  cormaster                                              
  cyberball                 origin:atari                 
  dambor                    name:gborges                 
  default                                                
  demo-cluster-group                                     
  devnexus2020                                           
  dfleming    

Notice above the cluster group "cg-jbrightwell" does not exist

In [4]:
!tmc cluster list

  NAME                           CLUSTERGROUP           PROVIDER        STATE     HEALTH        LABELS                                                 
  allure                         rccl                   UNKNOWN         PENDING   NA            tmc.cloud.vmware.com/creator:makrishna                 
  arao-kind                      pa-arao                UNKNOWN         PENDING   NA            tmc.cloud.vmware.com/creator:arao_pivotal.io           
  aro-tmc-cluster-1              pa-arao                AWS_EC2         UNKNOWN   DISCONNECTED  tmc.cloud.vmware.com/creator:arao_pivotal.io           
  arul-aws-east-prod             arul-tmc               AWS_EC2         UNKNOWN   DISCONNECTED  tmc.cloud.vmware.com/creator:avannala_pivotal.io       
  arul-azure-prod                arul-tmc               AZURE_COMPUTE   READY     UNHEALTHY     tmc.cloud.vmware.com/creator:avannala_pivotal.io       
  arul-gcp-central-prod          arul-tmc               GCP_GCE         UNKNOWN   DISCON

Notice above the cluster "jbrightwell-c04" does not exist

In [5]:
!tmc clustergroup templates list

  NAME         DESCRIPTION                                            
  default-iam  default template for the iam-policy resource           
  generated    generated full template for the ClusterGroup resource  
  default      A default ClusterGroup object.                         


## Create a new cluster group if desired

In [13]:
!tmc clustergroup create -n pa-jkhan-aeo -t default

ℹ using template "default"
✔ clustergroup "pa-jkhan-aeo" created successfully 


In [14]:
!tmc clustergroup list

  NAME                      LABELS                       
  akadari                   origin:akadari               
  area51                    origin:atari                 
  arul-tmc                                               
  astroids                  origin:atari                 
  avannala                                               
  bboe                                                   
  bragazzi-pa-pks                                        
  caas-workshop             team:dell-tech               
  cdelashmutt                                            
  centipede                 origin:atari                 
  cormaster                                              
  cyberball                 origin:atari                 
  dambor                    name:gborges                 
  default                                                
  demo-cluster-group                                     
  devnexus2020                                           
  dfleming    

## Attach the PKS cluster into the new Cluster Group

In [15]:
!tmc cluster attach -g pa-jkhan-aeo -n poway-pks-cluster-1

✔ cluster "poway-pks-cluster-1" created successfully 
ℹ Run `kubectl apply -f k8s-attach-manifest.yaml` to attach the cluster


In [16]:
!kubectl apply -f k8s-attach-manifest.yaml

namespace/vmware-system-tmc unchanged
configmap/stack-config configured
secret/tmc-client-secret configured
customresourcedefinition.apiextensions.k8s.io/agents.clusters.tmc.cloud.vmware.com configured
customresourcedefinition.apiextensions.k8s.io/extensions.clusters.tmc.cloud.vmware.com configured
serviceaccount/extension-manager configured
clusterrole.rbac.authorization.k8s.io/extension-manager-role configured
clusterrolebinding.rbac.authorization.k8s.io/extension-manager-rolebinding configured
service/extension-manager-service configured
deployment.apps/extension-manager configured
serviceaccount/extension-updater-serviceaccount configured
clusterrole.rbac.authorization.k8s.io/extension-updater-clusterrole configured
clusterrolebinding.rbac.authorization.k8s.io/extension-updater-clusterrolebinding configured
service/extension-updater configured
deployment.apps/extension-updater configured
serviceaccount/agent-updater configured
clusterrole.rbac.authorization.k8s.io/agent-updater-rol

In [17]:
!kubectl get namespaces

NAME                STATUS   AGE
default             Active   6d23h
kube-node-lease     Active   6d23h
kube-public         Active   6d23h
kube-system         Active   6d23h
pa-jkhan-ns         Active   6d20h
pks-system          Active   6d23h
vmware-system-tmc   Active   6d21h


Notice the "vmware-system-tmc" namespace has been created.

In [18]:
!kubectl -n vmware-system-tmc get all

NAME                                           READY   STATUS      RESTARTS   AGE
pod/agent-updater-7b47bd5d65-hwjs5             1/1     Running     0          6d4h
pod/agentupdater-workload-1584310440-5hmmk     0/1     Completed   0          40s
pod/cluster-health-extension-6999fbdff-gp77v   1/1     Running     0          6d4h
pod/data-protection-5b94df79f4-5ckcd           1/1     Running     0          6d21h
pod/extension-manager-77c5bb9f89-rb45d         1/1     Running     0          6d4h
pod/extension-updater-8b464c8cf-6vbhg          1/1     Running     0          6d21h
pod/inspection-extension-6dbd8674fb-8xv99      1/1     Running     0          6d4h
pod/intent-agent-68bf74f8ff-4hqj7              1/1     Running     0          6d4h
pod/policy-sync-extension-54fdccd7bb-77n9q     1/1     Running     0          6d4h
pod/policy-webhook-64866489bd-84hlj            1/1     Running     0          6d21h
pod/policy-webhook-64866489bd-9ks7f            1/1     Running     0          6d21h
po

In [19]:
!tmc cluster list --group pa-jkhan-aeo

  NAME                 CLUSTERGROUP  PROVIDER  STATE    HEALTH  LABELS                                         
  poway-pks-cluster-1  pa-jkhan-aeo  UNKNOWN   PENDING  NA      tmc.cloud.vmware.com/creator:jkhan_pivotal.io  


Here are the templates to use to create the Workspace

In [20]:
!tmc workspace templates list

  NAME         DESCRIPTION                                         
  generated    generated full template for the Workspace resource  
  default      A default Workspace object.                         
  default-iam  default template for the iam-policy resource        


In [22]:
!tmc workspace create -n pa-jkhan-aeo-ws -d "app" -d default

ℹ using template "default"
✔ workspace "pa-jkhan-aeo-ws" created successfully 


In [25]:
!tmc cluster namespace create -c poway-pks-cluster-1 -n pa-jkhan-aeo -d "application" -k pa-jkhan-aeo-ws

ℹ using template "default"
✔ namespace "pa-jkhan-aeo" created successfully 


In [35]:
!kubectl get namespace

NAME                STATUS   AGE
default             Active   6d23h
kube-node-lease     Active   6d23h
kube-public         Active   6d23h
kube-system         Active   6d23h
pa-jkhan-ns         Active   6d20h
pks-system          Active   6d23h
vmware-system-tmc   Active   6d21h
